In [5]:
%pip install pandas ccxt yfinance numpy


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
from datetime import datetime

import ccxt
import yfinance as yf

In [7]:
input_train_file_path = '../data/train_targets.parquet'
output_historical_data_file_path = '../data/historical_data.csv'

train_df = pd.read_parquet(input_train_file_path)

In [8]:
# Load your dataset
data = pd.read_parquet(input_train_file_path)

# Initialize the ccxt binance client
exchange = ccxt.binance()

# Define a function to fetch historical data from Binance
def fetch_historical_data_binance(symbol, start_date, end_date):
    try:
        since = exchange.parse8601(start_date + 'T00:00:00Z')
        
        # Fetch daily OHLCV data
        ohlcv = exchange.fetch_ohlcv(symbol, '1d', since)
        
        # Convert to DataFrame
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['date'] = pd.to_datetime(df['timestamp'], unit='ms').dt.date
        return df[['date', 'open', 'high', 'low', 'close', 'volume']]
    except Exception as e:
        #print(f"Error fetching data for {symbol} from {start_date} to {end_date}: {e}")
        return None

# Define a function to fetch historical data from Yahoo Finance
def fetch_historical_data_yahoo(symbol, start_date, end_date):
    try:
        ticker = yf.Ticker(symbol)
        historical_data = ticker.history(start=start_date, end=end_date)
        
        historical_data.reset_index(inplace=True)
        historical_data['date'] = historical_data['Date'].dt.date
        return historical_data[['date', 'Open', 'High', 'Low', 'Close', 'Volume']].rename(columns={
            'Open': 'open',
            'High': 'high',
            'Low': 'low',
            'Close': 'close',
            'Volume': 'volume'
        })
    except Exception as e:
        print(f"Yahoo: Error fetching data for {symbol} from {start_date} to {end_date}: {e}")
        return None

# Set the date range
start_date = data['date'].min()
end_date = datetime.now().date() # Use the end date as the current date

print(f'Fetching data from {start_date} till {end_date}')

# Fetch data for all symbols
all_data = pd.DataFrame()
for symbol in data['symbol'].unique():
    symbol_binance = symbol + '/USDT'
    symbol_yahoo = symbol + '-USD'
    
    # Try fetching historical data from Binance
    historical_data = fetch_historical_data_binance(symbol_binance, start_date, end_date)
    
    # If Binance data is not available, use Yahoo Finance as fallback
    if historical_data is None:
        historical_data = fetch_historical_data_yahoo(symbol_yahoo, start_date, end_date)
    
    # Add the symbol to the data
    if historical_data is not None and not historical_data.empty:
        historical_data['symbol'] = symbol
        all_data = pd.concat([all_data, historical_data])

# Save the fetched data to a CSV file
all_data.to_csv(output_historical_data_file_path, index=False)
print(f'Done! All the data has been downloaded and saved here: {output_historical_data_file_path}')

Fetching data from 2020-06-01 till 2024-08-01
Yahoo: Error fetching data for STRX-USD from 2020-06-01 to 2024-08-01: index 0 is out of bounds for axis 0 with size 0


$METAL-USD: possibly delisted; No timezone found


Yahoo: Error fetching data for METAL-USD from 2020-06-01 to 2024-08-01: Can only use .dt accessor with datetimelike values


$$MONG-USD: possibly delisted; No timezone found


Yahoo: Error fetching data for $MONG-USD from 2020-06-01 to 2024-08-01: Can only use .dt accessor with datetimelike values
